In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import minmax_scale

In [ ]:
##### Deseq2_LRT_median ratios nomalization_DEG
file = './source_data/DEmRNA/batch_cor_Gene_LRT_normalized_count_TC_mfa.txt'
DEG_norm_count_mfa = pd.read_csv(file, sep = '\t', header = 0)
DEG_norm_count_mfa = DEG_norm_count_mfa.reset_index(drop=False)
DEG_norm_count_mfa = DEG_norm_count_mfa.rename(columns={'index':'gene_chr'})
DEG_norm_count_mfa= DEG_norm_count_mfa.set_index('gene_chr')

##### Gene Ontology Genes
path_dir = './source_data/GOterm'
file_list = os.listdir(path_dir)
    
GO = pd.DataFrame(columns=['source','id','gene','NONE','annotation_class','reference',
                           'evidence','evidence_with','qualifier','gene_name',
                           'synonym','type','isoform','date','contributor','NONE','NONE','Group'])

for i in file_list:
    #print(i)
    if i.startswith('AmiGo2') and i.endswith('txt'):
        file = './source_data/GOterm/%s' % i
        x = pd.read_csv(file,sep='\t',header = None)
        x.columns=['source','id','gene','NONE','annotation_class','reference',
                   'evidence','evidence_with','qualifier','gene_name',
                   'synonym','type','isoform','date','contributor','NONE','NONE']
        x['Group'] = (i.split('.')[0]).split('_')[1]
        #print(x.shape)
        
        GO = pd.concat([GO,x],axis=0)

GO = GO[['gene','source','annotation_class','Group']]
GO['gene'] = GO['gene'].apply(lambda x: x.upper())

In [ ]:
print(GO.shape)
#GO = GO[GO['source']=='UniProtKB']
print(GO.shape)

In [ ]:
def file_parsing_mfa(data):

    data = data.reset_index(drop=False)
    data = data.rename(columns={'index':'gene_chr'})
    data_melt = pd.melt(data, id_vars=['gene_chr'])
    conditionlist = [data_melt.variable == 'A_FST',
                     data_melt.variable == 'B_FST',
                     data_melt.variable == 'A_SND',
                     data_melt.variable == 'B_SND',
                     data_melt.variable == 'A_TRD',
                     data_melt.variable == 'B_TRD',
                 
                     data_melt.variable == 'C_FST',
                     data_melt.variable == 'D_FST',
                     data_melt.variable == 'C_SND',
                     data_melt.variable == 'D_SND',
                     data_melt.variable == 'C_TRD',
                     data_melt.variable == 'D_TRD',
                 
                     data_melt.variable == 'E_FST',
                     data_melt.variable == 'F_FST',
                     data_melt.variable == 'E_SND',
                     data_melt.variable == 'F_SND',
                     data_melt.variable == 'E_TRD',
                     data_melt.variable == 'F_TRD',
                 
                     data_melt.variable == 'G_FST',
                     data_melt.variable == 'H_FST',
                     data_melt.variable == 'G_SND',
                     data_melt.variable == 'H_SND',
                     data_melt.variable == 'G_TRD',
                     data_melt.variable == 'H_TRD']
    choicelist = [3,3,4,4,5,5,7,7,8,8,9,9,12,12,13,13,14,14,16,16,17,17,18,18]
    data_melt['age'] = np.select(conditionlist,choicelist, default='Not Specified')


    conditionlist = [data_melt.variable.str.split('_').str[0] == 'A',
                     data_melt.variable.str.split('_').str[0] == 'B',
                     data_melt.variable.str.split('_').str[0] == 'C',
                     data_melt.variable.str.split('_').str[0] == 'D',
                     data_melt.variable.str.split('_').str[0] == 'E',
                     data_melt.variable.str.split('_').str[0] == 'F',
                     data_melt.variable.str.split('_').str[0] == 'G',
                     data_melt.variable.str.split('_').str[0] == 'H']
    choicelist = ['G1','G1','G2','G2','G3','G3','G4','G4']
    choicelist_N = [1,1,2,2,3,3,4,4]
    data_melt['Group'] = np.select(conditionlist,choicelist, default='Not Specified')
    data_melt['group'] = np.select(conditionlist,choicelist_N, default='Not Specified')

    data_melt['value'] = data_melt['value'].round()
    data_melt['value'] = pd.to_numeric(data_melt['value'])
    data_melt['age'] = pd.to_numeric(data_melt['age'])
    
    return data_melt


def file_parsing_mml(data):
    
    GeneCPM_MML = data.reset_index(drop=False)
    GCL = pd.melt(GeneCPM_MML, id_vars=['gene_chr'])
    GCL = GCL.rename(columns={'variable':'sample'})
    GCL = pd.merge(GCL,condition,how='inner',on='sample')
    
    conditionlist = [(5 >= GCL['age']) & (GCL['age'] >= 4),
                 (11 >= GCL['age']) & (GCL['age'] >= 7),
                 (19 >= GCL['age']) & (GCL['age'] >= 15),
                 (22 >= GCL['age']) & (GCL['age'] >= 21)]
    choicelist = ['G1','G2','G3','G4']
    GCL['Group'] = np.select(conditionlist,choicelist, default='Not Specified')
                             
    return GCL

def score(df):
     
    DF = pd.DataFrame()
    for i in df['gene'].unique():
        G = df[df['gene'] == i]       
        G = G.copy()
        G['minmax'] = minmax_scale(G['value'])
         
        DF = pd.concat([DF,G], axis=0)

    return DF

In [ ]:
mfa = file_parsing_mfa(DEG_norm_count_mfa)
mfa['gene'] = mfa.gene_chr.str.split('_').str[0]

In [ ]:
def score_plot(LIST,column,Tit):

    #TC=len(LIST)
    R=4
    C=column
    
    N=1
    
    plt.figure(figsize = (column*4,column*4))
    
    for i in LIST:

        #print(i)
        GeneList = GO[GO['Group'] == i]
        #print(len(GeneList['gene'].unique())) 
        Lst = GeneList['gene'].unique()
        G = mfa[mfa['gene'].isin(Lst)]
        G = G.sort_values(by=['group'], ascending=[True])
        G = score(G)
        #print(len(G['gene'].unique()))
        G = G.groupby(['variable','age','Group'])['minmax'].mean()
        G = pd.DataFrame(G)
        G = G.reset_index(drop=False)
    
    
        plt.subplot(R,C,N)
        plt.gca().spines['right'].set_visible(False)
        plt.gca().spines['left'].set_visible(True)
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['bottom'].set_visible(True)
        
        PAL = ['#ED8080' if i == 'G1' else '#90EE90' if i == 'G2' else '#ADD8E6' if i == 'G3' else '#EFBBF0' for i in G['Group'].unique()]
        sns.boxplot(data=G,x='Group',y='minmax', palette=PAL,showfliers=True)
        plt.title(i)
        plt.xlabel('')
        plt.ylabel('')
        
        N+=1
    
    #plt.rcParams["figure.autolayout"] = True
    #plt.tight_layout()
    
    #plt.savefig('./'+Tit+'_'+'_expression.png',dpi=500)
    plt.show()

In [ ]:
L = ['phagocytosis, engulfment','phagolysosome assembly','phagocytosis, recognition','apoptotic cell clearance']
score_plot(L,4,'phagocytosis')

In [ ]:
L = ['neutrophil chemotaxis','eosinophil chemotaxis','basophil chemotaxis',
     'monocyte chemotaxis','macrophage chemotaxis',
     'B cell chemotaxis','T cell chemotaxis','T-helper 17 cell chemotaxis','dendritic cell chemotaxis']
score_plot(L,4,'chemotaxis')

In [ ]:
L = ['neutrophil activation','monocyte activation','macrophage activation','natural killer cell activation',
     'T cell activation','B cell activation','platelet activation']
score_plot(L,4,'Cellactivation')

In [ ]:
L = ['CD4-positive, alpha-beta T cell activation','T-helper 1 cell differentiation','T-helper 2 cell differentiation',
     'T-helper 17 cell differentiation','regulatory T cell differentiation','NK T cell activation',
     'CD8-positive, alpha-beta T cell activation','memory T cell activation','gamma-delta T cell activation']
score_plot(L,4,'Tcell')

In [ ]:
L = ['pre-B cell differentiation','transitional one stage B cell differentiation','transitional one stage B cell differentiation','follicular B cell differentiation',
     'germinal center B cell differentiation','marginal zone B cell differentiation','memory B cell differentiation','plasma cell differentiation']
score_plot(L,4,'Bcell')

In [ ]:
L = ['immunoglobulin V(D)J recombination','T cell receptor V(D)J recombination']
score_plot(L,4,'V(D)J recombination')

In [ ]:
L = ['homologous recombination','mitotic recombination','recombinational repair']
score_plot(L,4,'recombination')

In [ ]:
L = ['base-excision repair','double-strand break repair','mismatch repair','single strand break repair',
     'UV-damage excision repair','pyrimidine dimer repair','protein-DNA covalent cross-linking repair',
     'postreplication repair','nucleotide-excision repair','interstrand cross-link repair',
     'recombinational interstrand cross-link repair']
score_plot(L,4,'recombination_repair')

In [ ]:
L = ['inflammatory response','inflammatory response to antigenic stimulus',
     'inflammatory response to wounding','acute inflammatory response']
score_plot(L,4,'recombination_repair')

In [ ]:
L = ['leukocyte mediated cytotoxicity','natural killer cell mediated cytotoxicity','neutrophil mediated cytotoxicity','T cell mediated cytotoxicity']
score_plot(L,4,'cytotoxicity')